In [1]:
import numpy as np
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
import tscv

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
X = np.random.rand(100, 10)
y = np.random.rand(100)

In [3]:
# Define custom preprocessor
class StandardScaler(tscv.Preprocessor):
    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.std_ = np.std(X, axis=0)

    def transform(self, X):
        return (X - self.mean_) / self.std_

In [4]:
# Define custom model
class CustomLinearModel(tscv.Model):
    def fit(self, X, y):
        self.coef_ = np.linalg.pinv(X) @ y

    def predict(self, X):
        return X @ self.coef_

In [10]:
# Define custom metrics
class MeanSquaredError(tscv.Metric):
    def evaluate(self, y_obs, y_pred):
        return np.mean((y_obs - y_pred) ** 2)
    
class MeanAbsoluteError(tscv.Metric):
    def evaluate(self, y_obs, y_pred):
        return np.mean(np.abs(y_obs - y_pred))

In [11]:
# Set up cross-validation
cross_validator = tscv.TimeSeriesCrossValidator(
    train_starts=[0, 20, 40],
    train_sizes=[15, 20, 25],
    test_sizes=[5, 5, 5],
    preprocessor=StandardScaler()
)

In [12]:
# Define model and preprocessor
model = CustomLinearModel()

In [13]:
# Define metrics
metrics = [MeanSquaredError(), MeanAbsoluteError()]

In [14]:
# Evaluate model
evaluations = cross_validator.cross_validate(model, X, y, metrics)
print("Evaluation Results:")
for evaluation in evaluations:
    print(evaluation)

Evaluation Results:
{'MeanSquaredError': 0.26955336904854693, 'MeanAbsoluteError': 0.4603207931241317}
{'MeanSquaredError': 0.12833842568255596, 'MeanAbsoluteError': 0.2883063441674924}
{'MeanSquaredError': 0.4047077594926084, 'MeanAbsoluteError': 0.602132051913923}
